In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

import pandas as pd

C:\Users\Vince\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# The complete works of Shakespeare in a single text file!
# (Technology is amazing, isn't it?)
text = io.open('shakespeare.txt', encoding='utf-8').read().lower()

In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# Allows us to convert characters to numbers, and vice-versa
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 58


In [19]:
# Put the text in semi-redundant sequences of maxlen characters
# (Best shown through example; see following cells)

maxlen = 14
step = 1

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 5447150


In [20]:
# Creating blank tensors that we'll fill in...
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


# But we need to convert the character to an index because we're one-hot encoding
# So we use the char_indices dict
# y[i, (char_index)] is the cell we need to one-hot encode
for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i, j, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Each data point in x is a one-hot-encoded sequence of length 40
print('x.shape:', x.shape)

# Each y is a one-hot-encoded next character
print('y.shape:', y.shape)

Vectorization...
x.shape: (5447150, 14, 58)
y.shape: (5447150, 58)


In [21]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce = ReduceLROnPlateau(factor=.01, patience=3, monitor='loss')
es = EarlyStopping(patience=5, monitor='loss')

In [22]:
# Build the model: a single LSTM
print('Building model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Building model...


In [27]:
try:
    model.fit(x, y,
              batch_size=256,
              epochs=1, callbacks=[es, reduce])
except KeyboardInterrupt:
    pass

Epoch 1/1
5447150/5447150 [==============================] - 1354s 249us/step - loss: 1.3691

In [28]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:
from warnings import filterwarnings
filterwarnings('ignore')

# Construct a "seed sentence", from the random point, of maxlen length
start_index = random.randint(0, len(text) - maxlen - 1)

# Keep a string of all generated words
allgen = ''

for diversity in [0.2, 0.5, 1.0, 1.2]:

    generated = ''

    sentence = text[start_index: start_index + maxlen]
    generated += sentence

    # We're going to sequentially build on this empty string (400 times)
    sys.stdout.write(generated)

    for i in range(1000):

        # This one-hot encodes the sentence we currently have
        # (Starting with the seed sentence)
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        # Predict the one-hot encoding we just made
        preds = model.predict(x_pred, verbose=0)[0]

        # Sample an index (diversity tries different temperatures)
        next_index = sample(preds, diversity)

        # Convert that index number to a character
        next_char = indices_char[next_index]

        # Add that character to our sentence
        generated += next_char
        allgen += next_char

        # Shift the sentence string by 1 to include our new, predicted character
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

garet. nay, good man and the man the state of the compantion and the present to the month of the words of the prince to a soldier, and but the steel of the strange the song of the world.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [30]:
model.save_weights('shakespeare.h5')